### <span style=color:blue> DISCUSSION 7 - Working with mongodb (local)    </span>

In [ ]:
import sys
import json
import csv
import yaml

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

import pprint

import psycopg2
from sqlalchemy import create_engine, text as sql_text

##### <span style=color:blue>Basic queries in mongoDB</span>

In [ ]:
# Setup the mongoclient and connect to local instance of mongodb
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")

In [ ]:
# Call a db company or create one if it does not exists
db = client["company"]

In [ ]:
# Insert a document
db.employees.insert_one({"name": "Alice", "age": 28, "department": "HR"})

In [ ]:
# Find a document
employee = db.employees.find_one({"name": "Alice"})
print(employee)

In [ ]:
# Update a document
db.employees.update_one({"name": "Alice"}, {"$set": {"age": 29}})

In [ ]:
# Delete a document
db.employees.delete_one({"name": "Alice"})

In [ ]:
# Find all employees in the HR department
employees_in_hr = db.employees.find({"department": "HR"})
for emp in employees_in_hr:
    print(emp)

# Count the number of employees in the company
employee_count = db.employees.count_documents({})
print(f"Number of employees: {employee_count}")


##### <span style=color:blue>Working with pre-existing data</span>

In [ ]:
# Load CSV files into DataFrames
department_df = pd.read_csv("data/department.csv")
dependent_df = pd.read_csv("data/dependent.csv")
dept_locations_df = pd.read_csv("data/dept_locations.csv")
employee_df = pd.read_csv("data/employee.csv")
project_df = pd.read_csv("data/project.csv")
works_on_df = pd.read_csv("data/works_on.csv")

print(department_df)
print(dependent_df)
print(dept_locations_df)
print(employee_df)
print(project_df)
print(works_on_df)

In [ ]:
# Insert data into MongoDB and check in MongoDB Compass
db.departments.insert_many(department_df.to_dict('records'))
db.dependents.insert_many(dependent_df.to_dict('records'))
db.dept_locations.insert_many(dept_locations_df.to_dict('records'))
db.employees.insert_many(employee_df.to_dict('records'))
db.projects.insert_many(project_df.to_dict('records'))
db.works_on.insert_many(works_on_df.to_dict('records'))

In [ ]:
# Intializing pprint 

pp = pprint.PrettyPrinter(indent=4)

In [ ]:
# Print all employees in company

print("Employees in the Database:")

employees = db.employees.find()
for emp in employees:
    pp.pprint(emp)

In [ ]:
# Find all employees in the "Sales" department

employees_in_sales = db.employees.find({"dno": 1})
for emp in employees_in_sales:
    pp.pprint(emp)

In [ ]:
# Combined query using aggregation pipeline to find the emplyees
# in the research department

research_department_employees = db.employees.aggregate([
    {
        # Performs a join between employees and departments on the dno field
        "$lookup": {
            "from": "departments",
            "localField": "dno",
            "foreignField": "dnumber",
            "as": "department_info"
        }
    },
    # Deconstructs the array field to output a document for each element
    {"$unwind": "$department_info"},
    
    # Filtering operation to include only dname=="Research"
    {"$match": {"department_info.dname": "Research"}}
])

# Print the results
for emp in research_department_employees:
    pp.pprint(emp)
    
# The above query will look like follows in SQL 
# SELECT e.*
# FROM employees e
# JOIN departments d ON e.dno = d.dno
# WHERE d.dname = 'Administration';

In [ ]:
# Query to find projects managed by the "Research" department

research_department_projects = db.projects.aggregate([
    {
        # Performs a join between employees and departments on the dno file 
        "$lookup": {
            "from": "departments",
            "localField": "dnum",
            "foreignField": "dnumber",
            "as": "department_info"
        }
    },
    # Deconstructs the array field to output a document for each element
    {"$unwind": "$department_info"},
    
    # Filtering operation to include only dname==Research
    {"$match": {"department_info.dname": "Research"}}
])

print("Projects Managed by Research Department:")

for proj in research_department_projects:
    pp.pprint(proj)

##### <span style=color:blue>Aggregation Queries</span>

In [ ]:
# Calculate the average salary in each department

average_salary_per_department = db.employees.aggregate([
    {
        # Group documents by the dno field and create a new average_salary field
        "$group": {
            "_id": "$dno",
            "average_salary": {"$avg": "$salary"}
        }
    },
    {
        # Perform join between departments and employees
        "$lookup": {
            "from": "departments",
            "localField": "_id",
            "foreignField": "dnumber",
            "as": "department_info"
        }
    },
    {"$unwind": "$department_info"},
    {
        # Shape the output documents to include only the required fields.
        "$project": {
            "department_name": "$department_info.dname",
            "average_salary": 1
        }
    }
])

print("Average Salary Per Department:")

for dept in average_salary_per_department:
    pp.pprint(dept)

# The above query will look like follows in SQL
# SELECT 
#     d.dname AS department_name,
#     AVG(e.salary) AS average_salary
# FROM 
#     employees e
# JOIN 
#     departments d ON e.dno = d.dnumber
# GROUP BY 
#     d.dname;

In [ ]:
# Aggregation pipeline to find the total hours worked on each project
total_hours_per_project = db.works_on.aggregate([
    {
        "$group": {
            "_id": "$pno",
            "total_hours": {"$sum": "$hours"}
        }
    },
    {
        "$lookup": {
            "from": "projects",
            "localField": "_id",
            "foreignField": "pnumber",
            "as": "project_info"
        }
    },
    {"$unwind": "$project_info"},
    {
        "$project": {
            "project_name": "$project_info.pname",
            "total_hours": 1
        }
    }
])

print("Total Hours Worked on Each Project:")

for project in total_hours_per_project:
    pp.pprint(project)
    
# The above query will look like follows in SQL
# SELECT 
#     p.pname AS project_name,
#     SUM(COALESCE(w.hours, 0)) AS total_hours
# FROM 
#     works_on w
# JOIN 
#     projects p ON w.pno = p.pnumber
# GROUP BY 
#     p.pname;

In [ ]:
# Update all documents in works_on where hours is NaN to set hours to 0
result = db.works_on.update_many(
    {"hours": {"$exists": True, "$type": "double", "$in": [float('nan')]}},
    {"$set": {"hours": 0}}
)

print(f"Matched {result.matched_count} documents and modified {result.modified_count} documents.")

In [ ]:
# Run the "Aggregation pipeline to find the total hours worked on each project" again
# And check if you're getting nan values now